In [ ]:
print(pd.read_excel('StockData/GPW/_2022-03-03_akcje.xls').head(1))

In [ ]:
from Other.imports import *
from Other.constants import *

try:
    conn = mysql.connector.connect(
        host="localhost",
        user="StockInsertion",
        password="StockDataInsertion",
        database="Stock"
    )
    if conn.is_connected():
        print("Connected")

    cursor = conn.cursor()

    with open(GPW_JSON, 'r') as file:
        config = json.load(file)

    for file_name in os.listdir(config['downloads_path']):
        if not file_name.endswith(('.xls', '.xlsx')):
            continue  # skip non-excel files
        
        file_path = os.path.join(config['downloads_path'], file_name)
        day_prices = pd.read_excel(file_path)
        
        for index, row in day_prices.iterrows():
        # -- DayValue Table
        # CREATE TABLE DayValue (
        #     ID INT AUTO_INCREMENT PRIMARY KEY,
        #     CompanyID INT NOT NULL,
        #     `Date` DATE NOT NULL,
        #     `OPEN` DECIMAL(10,2) NOT NULL,
        #     `CLOSE` DECIMAL(10,2) NOT NULL,
        #     `MIN` DECIMAL(10,2) NOT NULL,
        #     `MAX` DECIMAL(10,2) NOT NULL,
        #     Volume INT NOT NULL,
        #     Trades INT NOT NULL,
        #     Turnover DECIMAL(10,2) NOT NULL,

        #     FOREIGN KEY (CompanyID) REFERENCES Company(ID) ON DELETE CASCADE,
        #     UNIQUE (CompanyID, `Date`)
        # );

            select_query = "SELECT ID FROM Company WHERE Identifier = %s"

            cursor.execute(select_query, (row["Nazwa"],))
            company_ID = cursor.fetchone() 
            
            if company_ID:
                company_ID = company_ID[0]
                #TODO if data and company combination exsist overrte
                insert_query = """
                        INSERT INTO DayValue (CompanyID, `Date`, `OPEN`, `CLOSE`, `MIN`, `MAX`, Volume, Trades, Turnover)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE
                            `OPEN` = VALUES(`OPEN`),
                            `CLOSE` = VALUES(`CLOSE`),
                            `MIN` = VALUES(`MIN`),
                            `MAX` = VALUES(`MAX`),
                            Volume = VALUES(Volume),
                            Trades = VALUES(Trades),
                            Turnover = VALUES(Turnover)
                    """
                date_val = pd.to_datetime(row['Data']).strftime('%Y-%m-%d')
                cursor.execute(insert_query, (
                    company_ID,
                    date_val,  # Ensure this is in proper date format
                    float(row['Kurs otwarcia']),
                    float(row['Kurs zamknięcia']),
                    float(row['Kurs min']),
                    float(row['Kurs max']),
                    int(row['Wolumen']),
                    int(row['Liczba Transakcji']),
                    float(row['Obrót']) * 1000
                ))

                conn.commit()

            else:
                print(f"No company found with Identifier = {row['Nazwa']}")



except mysql.connector.Error as err:
    print(f"MySQL error: {err}")
except FileNotFoundError as fnf_err:
    print(f"File error: {fnf_err}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    try:
        if cursor:
            cursor.close()
        if conn.is_connected():
            conn.close()
        print("MySQL connection closed.")
    except NameError:
        pass


In [ ]:
cursor.close()
conn.close()

In [1]:
import mysql.connector
from mysql.connector import Error

def insert_stock_exchange():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",       # replace with your MySQL user
            password="Kanakan9824",   # replace with your password
            database="Stock"    # replace with your DB name (e.g., Stock)
        )
        if conn.is_connected():
            print("Connected to database")

        cursor = conn.cursor()

        insert_query = """
            INSERT INTO StockExchange (ExchangeName, Country, City)
            VALUES (%s, %s, %s)
        """

        data = ("GPW", "Polska", "Warszawa")

        cursor.execute(insert_query, data)
        conn.commit()
        print("Record inserted successfully")

    except Error as e:
        print(f"Error: {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection closed")

if __name__ == "__main__":
    insert_stock_exchange()


Connected to database
Record inserted successfully
MySQL connection closed


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}